In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds
print('done')


done


In [3]:
# as_supervised = True , means dataset directly (input, label) pairs return karega:,
""" else dictionary,example= {'image': <tf.Tensor: shape=(28, 28, 1), dtype=uint8, numpy=...>,
'label': <tf.Tensor: shape=(), dtype=int64, numpy=5>}
"""
# 
''' ''''

In [7]:
# imbd dataset (pre-defined dataset) ko load krna
(train_data , validate_data , test ) = tfds.load(name = 'imdb_reviews' , 
                                                 split = ('train[:60%]','train[60:]','test'),
                                                 as_supervised = True
                                                )

In [15]:
train_example_batch , train_labels_batch = next(iter(train_data.batch(10)))
#iter() function se dataset ka iterator banaya jata hai, 
# jo ek-ek batch return karega. ,,isme batch size 10 hai
# while 
# next() function iterator ka first batch return karega.
# Ye batch ek tuple hota hai: (inputs, labels), jisme:
# inputs → Training examples (features)
# labels → Unka corresponding output (class labels)
"""
Ye code dataset ke first 10 training examples & labels ko extract karta hai 
batch-wise training ke liye! , 
while 
train_example_batch --→ First 10 training examples (features)
train_labels_batch --→ First 10 labels (outputs)
"""
train_labels_batch
# 📌 1. Data Load & Preprocessing -> done

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [27]:
# 📌 2. Text Embeddings Using TensorFlow Hub

embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1" 
# txt data ko number format (embeddings) me convert karna zaroori hai.
hub_layer = hub.KerasLayer( embedding, 
                          input_shape = [] , 
                          dtype = tf.string ,
                          trainable = True)
print("Embedding Layer Loaded ✅")
"""  
jo strings ko numerical vectors me convert karta hai.
Agar trainable=True
hai to hum embeddings ko fine-tune bhi kar sakte hain,
warna sirf pre-trained vectors ka use hoga. 🚀
"""
#Vectors in ai represent 
# data numerically, enabling machines to process, compare, and learn patterns efficiently.  
hub_layer(train_example_batch[:3])
# means 
# First 3 training examples ko embedding layer se pass kar rahe hain 
# taaki unko numerical vectors me convert kiya ja sake.

Embedding Layer Loaded ✅


<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [17]:
# 📌 3. Model Create Karna (Using Keras Sequential API)
# ✅ Neural Network Model (Using TensorFlow Keras)
# ✅ Wrap in a Lambda Layer
wrapped_hub_layer = tf.keras.layers.Lambda(lambda x: hub_layer(x))
# ✅ Wrapping with Lambda solves this issue by ensuring it acts as a Keras Layer.

# ✅ Define Model
model = tf.keras.Sequential([
    wrapped_hub_layer,  # Wrapping solves the issue
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(8, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
# Text input → hub_layer → Embeddings
# Embeddings → Dense Layers (MLP) → Output Prediction
# Works fine inside Sequential() because of Lambda wrapping.
print('done')

done


In [21]:
# 📌 4. Model Compile Karna
model.compile(
    loss='binary_crossentropy',  # Kyuki yeh Binary Classification hai
    optimizer='adam',  # Adaptive learning ke liye Adam Optimizer
    metrics=['accuracy']  # Accuracy ko track karne ke liye
)
# Binary Crossentropy: Kyuki yeh binary classification problem hai.
# Adam Optimizer: Kyuki yeh best learning rate optimization karta hai.
print("Model Compiled Successfully ✅")

Model Compiled Successfully ✅


In [31]:
# 📌 5. Model Train Karna
# ✅ Train Model
# Ab model train karege IMDB dataset ke training data pe.
import tensorflow_datasets as tfds

(train_data, validation_data, test_data) = tfds.load(
    name="imdb_reviews",
    split=("train[:60%]", "train[60%:]", "test"),
    as_supervised=True
)

# ✅ Check if dataset is loaded
print("Train Data:", train_data)
print("Validation Data:", validation_data)
print("Test Data:", test_data)

history = model.fit(
    train_data.batch(512),  
    epochs=10,
    validation_data=validation_data.batch(512),  # ✅ Fixed variable name
    verbose=1
)
print("Model Training Done ✅")


Train Data: <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
Validation Data: <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
Test Data: <_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>
Epoch 1/10



30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 39ms/step - accuracy: 0.4986 - loss: 0.8691 - val_accuracy: 0.5550 - val_loss: 0.7087
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.5838 - loss: 0.6894 - val_accuracy: 0.6159 - val_loss: 0.6571
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.6294 - loss: 0.6489 - val_accuracy: 0.6393 - val_loss: 0.6371
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6440 - loss: 0.6340 - val_accuracy: 0.6481 - val_loss: 0.6263
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.6508 - loss: 0.6252 - val_accuracy: 0.6555 - val_loss: 0.6190
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 28ms/step - accuracy: 0.6577 - loss: 0.6190 - val_accuracy: 0.6609 - val_loss: 0.6134
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.6644 - loss: 0.6141 - val_accuracy: 0.6658 - val_loss: 0.6088
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 29ms/step - accuracy: 0.6693 - loss: 0.6099 - val_accuracy: 0.6706 - val_loss: 0.

In [33]:
# 📌 6. Model Test Karna
# Model ko ab test dataset pe check karege.
# Agar test accuracy 80-90% tak hai toh model sahi train hua hai. ✅
test_loss, test_acc = model.evaluate(test_data.batch(512))
print(f'Test Accuracy: {test_acc:.4f}')


49/49 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6677 - loss: 0.6077
Test Accuracy: 0.6720


In [37]:
model.save("interview_ai_model.h5")
print("Model Saved Successfully ✅")
# H5 Format: Yeh TensorFlow ka standard format hai
# jo Android App ya Web App me use ho sakta hai.

Model Saved Successfully ✅


In [ ]:
sample_review = tf.convert_to_tensor(["This movie was absolutely fantastic! , Highly recommended!"], dtype=tf.string)  # ✅ Convert to tf.string tensor
prediction = model.predict(sample_review)

print(f"Sentiment Score: {prediction[0][0]}")